In [46]:
import numpy as np  
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
import itertools

In [25]:
train_data=pd.read_csv(r'../playground-series-s5e10/train.csv')
test_data = pd.read_csv(r'test.csv')

In [26]:
train_data.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [27]:
test_data.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3


In [28]:
train_data.columns
#features=['id','路况','路道数','路面弯曲','速度限制','光照状况','天气','道路信号'
#         ,'是否为公共道路','时间','是否为节假日','是否为上学季','报道事故数','交通风险']
#df.rename(columns=dict(zip(df.columns,features)),inplace=True)

Index(['id', 'road_type', 'num_lanes', 'curvature', 'speed_limit', 'lighting',
       'weather', 'road_signs_present', 'public_road', 'time_of_day',
       'holiday', 'school_season', 'num_reported_accidents', 'accident_risk'],
      dtype='object')

In [29]:
train_data.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [30]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
id,517754.0,258876.500000,149462.849975,0.0,129438.25,258876.50,388314.75,517753.0
num_lanes,517754.0,2.491511,1.120434,1.0,1.00,2.00,3.00,4.0
curvature,517754.0,0.488719,0.272563,0.0,0.26,0.51,0.71,1.0
speed_limit,517754.0,46.112575,15.788521,25.0,35.00,45.00,60.00,70.0
num_reported_accidents,517754.0,1.187970,0.895961,0.0,1.00,1.00,2.00,7.0
accident_risk,517754.0,0.352377,0.166417,0.0,0.23,0.34,0.46,1.0


In [31]:
train_data.road_type.unique()

array(['urban', 'rural', 'highway'], dtype=object)

In [32]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      517754 non-null  int64  
 1   road_type               517754 non-null  object 
 2   num_lanes               517754 non-null  int64  
 3   curvature               517754 non-null  float64
 4   speed_limit             517754 non-null  int64  
 5   lighting                517754 non-null  object 
 6   weather                 517754 non-null  object 
 7   road_signs_present      517754 non-null  bool   
 8   public_road             517754 non-null  bool   
 9   time_of_day             517754 non-null  object 
 10  holiday                 517754 non-null  bool   
 11  school_season           517754 non-null  bool   
 12  num_reported_accidents  517754 non-null  int64  
 13  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(

In [33]:
train_data.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [34]:
train_data.speed_limit.unique()
train_data.weather.unique()

array(['rainy', 'clear', 'foggy'], dtype=object)

In [35]:
## 编码
change_feature=['road_type','lighting','weather','time_of_day','holiday','school_season']
for col in change_feature:
    encoder=LabelEncoder()
    combine=pd.concat([train_data[col] , test_data[col]] ,axis=0).astype(str)
    encoder.fit(combine)
    train_data[col]=encoder.transform(train_data[col].astype(str))
    test_data[col]=encoder.transform(test_data[col].astype(str))

In [36]:
train_data.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,2,2,0.06,35,0,2,False,True,0,0,1,1,0.13
1,1,2,4,0.99,35,0,0,True,False,1,1,1,0,0.35
2,2,1,4,0.63,70,1,0,False,True,2,1,0,2,0.30
3,3,0,4,0.07,35,1,2,True,True,2,0,0,1,0.21
4,4,1,1,0.58,60,0,1,False,False,1,1,0,1,0.56


In [37]:
## 特征工程
for dataset in [train_data , test_data]:
    dataset["speed_curvature"] = dataset["speed_limit"] * dataset["curvature"]
    dataset["lane_density"] = dataset["num_lanes"] / (dataset["speed_limit"] + 1)
    dataset["accidents_per_lane"] = dataset["num_reported_accidents"] / (dataset["num_lanes"] + 1)
features = [f for f in train_data.columns if f not in ["id","accident_risk"]]

In [38]:
features

['road_type',
 'num_lanes',
 'curvature',
 'speed_limit',
 'lighting',
 'weather',
 'road_signs_present',
 'public_road',
 'time_of_day',
 'holiday',
 'school_season',
 'num_reported_accidents',
 'speed_curvature',
 'lane_density',
 'accidents_per_lane']

In [39]:
params_lgb = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.02,
    "num_leaves": 150,
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "random_state": 42,
    "n_estimators": 3000
}

params_cb = {
    "iterations": 3000,
    "learning_rate": 0.03,
    "depth": 8,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "random_seed": 42,
    "verbose": False
}

params_xgb = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "learning_rate": 0.03,
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "n_estimators": 3000,
    "random_state": 42,
    "early_stopping_rounds":100
}

In [40]:
folds=KFold(n_splits=5,shuffle=True,random_state=42)

In [41]:
oof_preds = {"lgb": np.zeros(len(train_data)), "cb": np.zeros(len(train_data)), "xgb": np.zeros(len(train_data))}
test_preds = {"lgb": np.zeros(len(test_data)), "cb": np.zeros(len(test_data)), "xgb": np.zeros(len(test_data))}

In [42]:
for fold_idx, (train_idx, valid_idx) in enumerate(folds.split(train_data)):
    print(f"\n🌿 Fold {fold_idx + 1}")
    
    X_tr, X_val = train_data.iloc[train_idx][features], train_data.iloc[valid_idx][features]
    y_tr, y_val = train_data.iloc[train_idx]["accident_risk"], train_data.iloc[valid_idx]["accident_risk"]
    
    # LightGBM
    model_lgb = lgb.LGBMRegressor(**params_lgb)
    model_lgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
    )
    oof_preds["lgb"][valid_idx] = model_lgb.predict(X_val)
    test_preds["lgb"] += model_lgb.predict(test_data[features]) / folds.n_splits

    # CatBoost
    model_cb = CatBoostRegressor(**params_cb)
    model_cb.fit(
        X_tr, y_tr,
        eval_set=(X_val, y_val),
        use_best_model=True,
        early_stopping_rounds=100
    )
    oof_preds["cb"][valid_idx] = model_cb.predict(X_val)
    test_preds["cb"] += model_cb.predict(test_data[features]) / folds.n_splits

    # XGBoost
    model_xgb = XGBRegressor(**params_xgb)
    model_xgb.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    oof_preds["xgb"][valid_idx] = model_xgb.predict(X_val)
    test_preds["xgb"] += model_xgb.predict(test_data[features]) / folds.n_splits



🌿 Fold 1
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002430 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 15
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fr

In [43]:
train_stack = pd.DataFrame({k: v for k, v in oof_preds.items()})
test_stack = pd.DataFrame({k: v for k, v in test_preds.items()})

In [44]:
def build_interaction_features(df):
    out = df.copy()
    for c1, c2 in itertools.combinations(df.columns, 2):
        out[f"{c1}_x_{c2}"] = df[c1] * df[c2]
    out["stack_mean"] = df.mean(axis=1)
    out["stack_std"] = df.std(axis=1)
    return out

train_ext = build_interaction_features(train_stack)
test_ext = build_interaction_features(test_stack)

print(f"\n🧱 Base models: {train_stack.shape[1]}")
print(f"🧮 Extended meta features: {train_ext.shape[1]}")



🧱 Base models: 3
🧮 Extended meta features: 8


In [48]:
ridge_alphas = [1e-3, 1e-2, 0.05, 0.1, 0.3, 1.0, 3.0, 10.0]
meta_model = RidgeCV(alphas=ridge_alphas, scoring="neg_root_mean_squared_error", cv=5)
meta_model.fit(train_ext, train_data["accident_risk"])

oof_final = meta_model.predict(train_ext)
pred_final = meta_model.predict(test_ext)

rmse_score = root_mean_squared_error(train_data["accident_risk"], oof_final)
print("\n=================================")
print(f"🔸 Final Stacking RMSE: {rmse_score:.5f}")
print(f"🔸 Best Alpha: {meta_model.alpha_}")
print("=================================")

# ============================================================
# 📤 Create Submission
# ============================================================
submission = pd.DataFrame({
    "id": test_data["id"],
    "accident_risk": pred_final
})
submission.to_csv("submission.csv", index=False)
print("\n✅ submission.csv successfully saved!")


🔸 Final Stacking RMSE: 0.05600
🔸 Best Alpha: 0.01

✅ submission.csv successfully saved!
